In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF']='max_split_size_mb:128'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from src.training.training_utils import *

import transformers

transformers.logging.set_verbosity_debug()

In [ ]:
def main():
    parquet_dir = '/workspace/data'
    stream_data = False
    base_model_id = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
    
    cpu_count = os.cpu_count()
    gpu_count = torch.cuda.device_count()
    
    print("Setting up training environment...")
    accelerator = setup_training_env()
    data_collator = create_data_collator(base_model_id)
    accelerator.print("Done setting up training environment...")
    
    accelerator.print(f"Number of CPUs available: {cpu_count}")
    accelerator.print(f"Number of GPUs available: {gpu_count}")
    
    accelerator.print("Loading data...")
    train_data, val_data, train_count, val_count = load_data(parquet_dir, stream=stream_data, cpu_count=cpu_count, accelerator=accelerator)
    accelerator.print("Done loading data...")
    
    accelerator.print("Loading compute_bertscore...")
    # compute_bertscore = load_bertscore()
    accelerator.print("Done loading compute_bertscore...")

    # x = train_data.select(range(100))
    
    # def test(a):
    #     return {
    #         'attention_mask': a['attention_mask'][:512],
    #         'input_ids': a['input_ids'][:512]
    #     }
    
    # x = x.map(test)
    
    accelerator.print("Loading model trainer...")
    trainer = load_model_trainer(
            base_model_id,
            False,
            data_collator,
            train_data,
            val_data,
            train_count,
            cpu_count//gpu_count,
            accelerator
        )
    
    print_gpu_utilization(accelerator)

    train_result = trainer.train()

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(main, num_processes=2)